<a href="https://colab.research.google.com/github/Alya109/transcriptSystem/blob/main/TranscriptNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("PLD Kups")

PLD Kups
